# Schnorr Signatures

In [1]:
import random
from IPython.display import Markdown as md

We start by defining Schnorr signatures on a cyclic group of prime order 23. This group may have come from an elliptic curve.

These signatures will not be strong! However, the implementation is simple for the purpose of making each calculation completely understandable.

In [2]:
p = 23
points = {i: chr(97 + i) for i in range(p)}

Private keys are numbers such that $0 \le n < p$

In [3]:
x_a = 17    # random.randint(0, p)
x_b = 15    # random.randint(0, p)

A = points[x_a]
B = points[x_b]

In [4]:
md(f'$(x_a, A)=({x_a}, {A})$')

$(x_a, A)=(17, r)$

In [5]:
md(f'$(x_b, B)=({x_b}, {B})$')

$(x_b, B)=(15, p)$

## Creating a hash function

In [6]:
def H(message: str, p: int = p):
    """A bad hash function for Z/23Z
    
    In general mod prime is not too bad, but
    since the alphabet only has 26 letters we
    might run into some collisions.
    Also it will be pretty clear how the function
    changes when we flip a letter.
    """
    res = sum([ord(c) for c in message])
    
    return res % p

In [7]:
message1 = 'Bitcoin Transaction'
print(f'MESSAGE: "{message1}", HASH: {H(message1)}')

message2 = 'Bitcoin transaction'
print(f'MESSAGE: "{message2}", HASH: {H(message2)}')

MESSAGE: "Bitcoin Transaction", HASH: 16
MESSAGE: "Bitcoin transaction", HASH: 2


## Signature function

In [8]:
def sign(m: str, sk: int, p: int = p):
    r = random.randint(0, 22)
    print(f'Random nonce for signature: {r}')
    return (r + H(m)*sk) % p, points[r]

Computing random signatures:

In [9]:
message = 'Bitcoin Transaction'
s_a, R_a = sign(message, x_a)
s_b, R_b = sign(message, x_b)

Random nonce for signature: 1
Random nonce for signature: 20


Our group is actually a 1-dimensional vector space over $F_{23}$.

Elements in $F_{23}$ are scalars, so let's define scalar multiplication:

In [10]:
def add_points(a: chr, b: chr, p: int = p):
    """This only works because we constructed our group
    in such a way that we know the discreet logarithm. In
    general, this will NOT work and is not a safe way to
    do this."""
    a = ord(a) - 97
    b = ord(b) - 97
    i = (a + b) % p
    return points[i]

def scalar_mult_point(scalar: int, point: int):
    scalar = scalar or p    # The scalar multiple should not be 0
    res = point
    for i in range(scalar - 1):
        res = add_points(res, point)
    return res

## Verification function

In [11]:
def verify(s, R, m, pup):
    lhs = points[s]
    rhs = add_points(R, scalar_mult_point(H(m), pup))
    return lhs == rhs

Verifying real signatures:

In [12]:
verify(s_a, R_a, message, A)

True

In [13]:
verify(s_b, R_b, message, B)

True

*Not* verifying false signatures:

In [14]:
verify((s_a + 1) % p, R_a, message, A)

False

In [15]:
verify(s_a, add_points(R_a, 'c'), message, A)

False

## MuSig Case

Note that you should just be able to add signatures:

$s_{a}=k_{a}+H(m)d_{a}$

$s_{b}=k_{b}+H(m)d_{b}$

$\Rightarrow s_{ab} =(s_{a}+s_{b})=(k_{a}+k_{b}) + H(m)(d_{a}+d_{b})$

$\Rightarrow s_{ab} = k_{ab} + H(m)d_{ab}$

In [16]:
md(f'A={A} and B={B}')

A=r and B=p

In [17]:
AB = add_points(A, B)
md(f'AB={AB}')

AB=j

In [18]:
s_ab = (s_a + s_b) % p
R_ab = add_points(R_a, R_b)

In [19]:
md('MuSig: $(s_{ab}, R_{ab})=$' + f' ({s_ab}, {R_ab}) is a signature for public key $AB={AB}$')

MuSig: $(s_{ab}, R_{ab})=$ (4, v) is a signature for public key $AB=j$

In [20]:
verify(s_ab, R_ab, message, AB)

True